### Dacon주관 금융문자분석 경진대회
[대회 개요](https://dacon.io/competitions/official/235401/overview/)

### import data

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import random
import pickle
import re

In [ ]:
import matplotlib.pyplot as plt
import pprint
from konlpy.tag import Okt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [306]:
test_  = pd.read_csv(r"C:\Users\wkd72\Desktop\19-2\filedown\public_test.csv", encoding='UTF-8')
train = pd.read_csv(r"C:\Users\wkd72\Desktop\19-2\filedown\train.csv", encoding='UTF-8')

In [1]:
path = "C:\\Users\\wkd72\\Desktop\\19-2\\공모전\\text\\filedown"
data = pd.read_csv("{}/train.csv".format(path))
data = data[["text", "smishing"]]
test = pd.read_csv("{}/public_test.csv".format(path))
test.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


### Train, Test set 비율 조정

In [6]:
random.seed(2050)
N= 20000
N_test = 2000
index = random.sample(range(1, data.shape[0]),N )
index_test = random.sample(range(1, data.shape[0]),N_test )

In [7]:
train = data.iloc[index,:].copy().reset_index(drop=True)
train_test = data.iloc[index_test,:].copy().reset_index(drop=True)

### 문장 데이터 전처리

In [9]:
#%%
def preprocessing(text_data, okt, remove_stopwords=False, stop_words=[]):
    '''
    remove_stopwords: 불용어 제거 여부 선택
    stop_words: 제거할 불용어 리스트
    '''
    text_data_refined = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text_data)
    text_data_okt = okt.pos(text_data_refined, stem=True)
    # 불용어 제거 부분 추가
    return text_data_okt

#%%
# =============================================================================
# 한글만을 추출
# =============================================================================
okt = Okt()
clean_text = []
#for i in range(len(train_data["text"])):
for i in range(N_test):
    text = train_test["text"][i]
    if type(text) == str: # 비어있는 데이터인지 확인
        clean_text.append(preprocessing(text, okt))
    else:
        clean_text.append([]) # 비어있는 데이터
    #printProgress(i, len(train_data["text"]) - 1, "preprocessing: ", "Complete", 1, 50)

#%%
# =============================================================================
# 특정 품사만을 선택
# =============================================================================
useful_tag = ['Noun', 'Verb', 'Adje', 'Adve', 'Suffix']
clean_testing = []
for i in range(len(clean_text)):
    clean_testing.append([x for (x, y) in clean_text[i] if y in useful_tag])

### 데이터 학습
> Simple LSTM 사용

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train)
sequences = tokenizer.texts_to_sequences(clean_train)

X_data = sequences
y_train = train['smishing']

max_len = 200
# 전체 데이터셋의 길이는 200으로 맞춥니다.
X_data = pad_sequences(X_data, maxlen=max_len,truncating = 'post')

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index)+1

model = Sequential()
model.add(Embedding(vocab_size, 32, input_length = max_len)) # 임베딩 벡터의 차원은 32
model.add(LSTM(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_data, y_train, epochs=3, batch_size=128, validation_split=0.25)

Instructions for updating:
Colocations handled automatically by placer.
Train on 15000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
15000/15000 [==============================] - 32s 2ms/sample - loss: 0.0933 - acc: 0.9821 - val_loss: 0.0137 - val_acc: 0.9990
Epoch 2/3
15000/15000 [==============================] - 30s 2ms/sample - loss: 0.0083 - acc: 0.9993 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 3/3
15000/15000 [==============================] - 30s 2ms/sample - loss: 0.0051 - acc: 0.9991 - val_loss: 0.0020 - val_acc: 0.9998


In [104]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_testing)
sequences = tokenizer.texts_to_sequences(clean_testing)

X_testing = sequences
pad_testing = pad_sequences(X_testing, maxlen=max_len, truncating = 'post')

In [105]:
X_test = pad_testing
y_test = train_test['smishing']

In [110]:
model.evaluate(X_test, y_test)[1]

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.2070 - acc: 0.9630


0.963

In [111]:
from sklearn import  metrics
result = model.predict(X_test)
print(metrics.roc_auc_score(y_test, result))

0.9514036016949153


### Test 데이터 예측

In [366]:
okt = Okt()
clean_text_test = []
#for i in range(len(train_data["text"])):
for i in range(test.shape[0]):
    text = test["text"][i]
    if type(text) == str: # 비어있는 데이터인지 확인
        clean_text_test.append(preprocessing(text, okt))
    else:
        clean_text_test.append([]) # 비어있는 데이터

In [367]:
useful_tag = ['Noun', 'Verb', 'Adje', 'Adve', 'Suffix']
clean_test = []
for i in range(len(clean_text_test)):
    clean_test.append([x for (x, y) in clean_text_test[i] if y in useful_tag])

In [368]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_test)
sequences = tokenizer.texts_to_sequences(clean_test)

In [369]:
word_to_index = tokenizer.word_index
#print(word_to_index)
vocab_size = len(word_to_index)+1
print(vocab_size)

X_data = sequences

5636


In [370]:
max_len = 150
# 전체 데이터셋의 길이는 200으로 맞춥니다.
data_test = pad_sequences(X_data, maxlen=max_len)

In [371]:
smishing_result = model.predict(data_test)
smishing_result = smishing_result.reshape(smishing_result.shape[0],)

In [372]:
smishing_result

array([0.00012481, 0.09851322, 0.0001353 , ..., 0.00015953, 0.00011337,
       0.00014754], dtype=float32)

In [373]:
result_excel = pd.DataFrame({'id' : test['id'], 'smishing' : smishing_result })

In [374]:
result_excel.to_csv("result.csv", index = False)